In [ ]:
!pip install -q langchain openai langchain-community sentence-transformers gradio requests faiss-cpu

In [ ]:
import gradio as gr
import os
import requests
from langchain.chat_models import ChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from google.colab import userdata
from langchain.docstore.document import Document
import shutil

In [ ]:
# --- Configuração da API Key do OpenRouter e TMDB ---
OPENROUTER_API_KEY = userdata.get('SUA_CHAVE_API_OPENROUTER')
os.environ["OPENAI_API_KEY"] = OPENROUTER_API_KEY
os.environ["OPENAI_API_BASE"] = "https://openrouter.ai/api/v1"

TMDB_API_KEY = userdata.get('SUA_CHAVE_API_TMDB')
TMDB_BASE_URL = "https://api.themoviedb.org/3"

# --- Inicializa o LLM da OpenRouter ---
llm = ChatOpenAI(
    model="deepseek/deepseek-chat-v3-0324:free",
    temperature=0.7,
    max_tokens=2000,
)

# --- Prompt Humanizado para Filmes e Séries (sem histórico de chat) ---
prompt_template = PromptTemplate(
    input_variables=["context", "question"], # Voltou a ser só context e question
    template="""
    Você é um assistente virtual especialista em filmes e séries.
    Seja gentil, educado e forneça informações detalhadas e precisas, sempre que possível.

    Sua principal função é **fornecer respostas objetivas e estritamente baseadas nas informações contidas no contexto fornecido**.
    Se a pergunta não puder ser respondida com as informações do contexto, responda educadamente que você não possui essa informação específica no momento e, se souber, sugira onde o usuário pode encontrar mais detalhes (ex: "Você pode tentar pesquisar em um site especializado em filmes.").
    **Não invente ou adivinhe respostas.**

    Contexto:
    {context}

    Pergunta:
    {question}

    Resposta:
    """
)

In [ ]:
# --- Função para buscar dados de filmes/séries na TMDB e criar Documentos ---
def fetch_media_data_as_documents(media_ids):
    documents = []
    print("Buscando dados de filmes e séries na TMDB...")
    for media_id in media_ids:
        media_type = None
        media_data = None

        # Tentar buscar como filme
        try:
            response = requests.get(f"{TMDB_BASE_URL}/movie/{media_id}", params={"api_key": TMDB_API_KEY, "language": "pt-BR"})
            response.raise_for_status()
            media_data = response.json()
            media_type = "Filme"
        except requests.exceptions.HTTPError as e:
            if e.response.status_code == 404:
                # Se não for encontrado como filme, tentar como série de TV
                try:
                    response = requests.get(f"{TMDB_BASE_URL}/tv/{media_id}", params={"api_key": TMDB_API_KEY, "language": "pt-BR"})
                    response.raise_for_status()
                    media_data = response.json()
                    media_type = "Série de TV"
                except requests.exceptions.RequestException as tv_e:
                    print(f"Erro ao buscar dados para ID {media_id} (filme ou série): {tv_e}")
                    continue # Pula para o próximo ID
            else:
                print(f"Erro ao buscar dados para ID {media_id} (filme): {e}")
                continue # Pula para o próximo ID
        except requests.exceptions.RequestException as e:
            print(f"Erro de conexão ao buscar dados para ID {media_id}: {e}")
            continue # Pula para o próximo ID

        if media_data:
            try:
                if media_type == "Filme":
                    title = media_data.get("title", "Título não disponível")
                    overview = media_data.get("overview", "Sinopse não disponível")
                    release_date = media_data.get("release_date", "Data de lançamento não disponível")
                    genres = ", ".join([g["name"] for g in media_data.get("genres", [])]) if media_data.get("genres") else "Gênero não disponível"
                    vote_average = media_data.get("vote_average", "Avaliação não disponível")
                    tagline = media_data.get("tagline", "")
                    runtime = media_data.get("runtime", "Duração não disponível")

                    # Buscar créditos (diretor, elenco principal)
                    credits_response = requests.get(f"{TMDB_BASE_URL}/movie/{media_id}/credits", params={"api_key": TMDB_API_KEY, "language": "pt-BR"})
                    credits_response.raise_for_status()
                    credits_data = credits_response.json()

                    director = "Diretor(es) não disponível(eis)"
                    for crew_member in credits_data.get("crew", []):
                        if crew_member.get("job") == "Director":
                            director = crew_member.get("name")
                            break

                    cast_members = [actor.get("name") for actor in credits_data.get("cast", [])[:5]] # Top 5 atores
                    cast = ", ".join(cast_members) if cast_members else "Elenco não disponível"

                    content = (
                        f"Tipo: {media_type}\n"
                        f"Título: {title}\n"
                        f"Sinopse: {overview}\n"
                        f"Slogan: {tagline}\n"
                        f"Data de Lançamento: {release_date}\n"
                        f"Gêneros: {genres}\n"
                        f"Duração: {runtime} minutos\n"
                        f"Avaliação Média: {vote_average}/10\n"
                        f"Diretor: {director}\n"
                        f"Elenco Principal: {cast}\n"
                        f"Detalhes adicionais: O {media_type.lower()} {title} é conhecido por sua {overview}. Lançado em {release_date}, ele se enquadra nos gêneros {genres} e possui uma avaliação de {vote_average}. A direção ficou por conta de {director} e conta com {cast} no elenco principal."
                    )
                    doc = Document(page_content=content, metadata={"source": f"TMDB {media_type} ID: {media_id}", "title": title, "type": media_type})
                    documents.append(doc)
                    print(f"Dados do {media_type.lower()} '{title}' (ID: {media_id}) processados.")

                elif media_type == "Série de TV":
                    title = media_data.get("name", "Título não disponível") # Para séries é 'name'
                    overview = media_data.get("overview", "Sinopse não disponível")
                    first_air_date = media_data.get("first_air_date", "Data de lançamento não disponível") # Para séries é 'first_air_date'
                    genres = ", ".join([g["name"] for g in media_data.get("genres", [])]) if media_data.get("genres") else "Gênero não disponível"
                    vote_average = media_data.get("vote_average", "Avaliação não disponível")
                    number_of_seasons = media_data.get("number_of_seasons", "Não disponível")
                    number_of_episodes = media_data.get("number_of_episodes", "Não disponível")

                    # Buscar créditos de séries (criadores, elenco principal)
                    credits_response = requests.get(f"{TMDB_BASE_URL}/tv/{media_id}/credits", params={"api_key": TMDB_API_KEY, "language": "pt-BR"})
                    credits_response.raise_for_status()
                    credits_data = credits_response.json()

                    creators = ", ".join([c["name"] for c in media_data.get("created_by", [])]) if media_data.get("created_by") else "Criador(es) não disponível(eis)"
                    cast_members = [actor.get("name") for actor in credits_data.get("cast", [])[:5]] # Top 5 atores
                    cast = ", ".join(cast_members) if cast_members else "Elenco não disponível"

                    content = (
                        f"Tipo: {media_type}\n"
                        f"Título: {title}\n"
                        f"Sinopse: {overview}\n"
                        f"Data de Lançamento: {first_air_date}\n"
                        f"Gêneros: {genres}\n"
                        f"Avaliação Média: {vote_average}/10\n"
                        f"Número de Temporadas: {number_of_seasons}\n"
                        f"Número de Episódios: {number_of_episodes}\n"
                        f"Criador(es): {creators}\n"
                        f"Elenco Principal: {cast}\n"
                        f"Detalhes adicionais: A {media_type.lower()} {title} é conhecida por sua {overview}. Lançada em {first_air_date}, ela se enquadra nos gêneros {genres} e possui uma avaliação de {vote_average}. Tem {number_of_seasons} temporadas e {number_of_episodes} episódios. Foi criada por {creators} e conta com {cast} no elenco principal."
                    )
                    doc = Document(page_content=content, metadata={"source": f"TMDB {media_type} ID: {media_id}", "title": title, "type": media_type})
                    documents.append(doc)
                    print(f"Dados da {media_type.lower()} '{title}' (ID: {media_id}) processados.")

            except Exception as e:
                print(f"Erro ao processar dados de {media_type} ID {media_id}: {e}")
                continue

    return documents

# --- IDs de filmes e séries para o exemplo ---
example_media_ids = [
    # Filmes - Já existentes
    157336,
    278,
    680,
    550,
    10227,
    496243,
    299534,
    872585,
    934640,
    671,
    603,
    19995,
    4935,

    # --- Novos Filmes Sugeridos ---
    11,
    12,
    13,
    18,
    24,
    429,
    447332,
    155,
    76341,
    122906,
    934640,
    968051,
    786891,
    573437,
    1011985,
    453395,
    823464,
    872585,
    1086747,
    653346,

    # --- Séries Sugeridas (IDs de TV shows na TMDB) ---
    1399,
    71712,
    1402,
    66732,
    92749,
    84958,
    119051,
    60735,
    1396,
    1668,
    456,
    94605,
    100088,
    2316,
    1400,
    690,
    45634,
]

all_documents = fetch_media_data_as_documents(example_media_ids)

if not all_documents:
    print("Nenhum documento de filme ou série foi carregado. Verifique sua chave da TMDB e IDs válidos.")
    all_documents.append(Document(page_content="Nenhum dado de filme ou série disponível para este chatbot. Por favor, configure a TMDB API key e IDs válidos.", metadata={"source": "Sistema"}))

Buscando dados de filmes e séries na TMDB...
Dados do filme 'Interestelar' (ID: 157336) processados.
Dados do filme 'Um Sonho de Liberdade' (ID: 278) processados.
Dados do filme 'Pulp Fiction: Tempo de Violência' (ID: 680) processados.
Dados do filme 'Clube da Luta' (ID: 550) processados.
Dados do filme 'Playtime - Tempo de Diversão' (ID: 10227) processados.
Dados do filme 'Parasita' (ID: 496243) processados.
Dados do filme 'Vingadores: Ultimato' (ID: 299534) processados.
Dados do filme 'Oppenheimer' (ID: 872585) processados.
Dados do filme 'الراهن الخاسر' (ID: 934640) processados.
Dados do filme 'Harry Potter e a Pedra Filosofal' (ID: 671) processados.
Dados do filme 'Matrix' (ID: 603) processados.
Dados do filme 'Avatar' (ID: 19995) processados.
Dados do filme 'O Castelo Animado' (ID: 4935) processados.
Dados do filme 'Guerra nas Estrelas' (ID: 11) processados.
Dados do filme 'Procurando Nemo' (ID: 12) processados.
Dados do filme 'Forrest Gump: O Contador de Histórias' (ID: 13) proce

In [ ]:
# --- Dividir documentos em chunks ---
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
docs = text_splitter.split_documents(all_documents)

# --- Configurando explicitamente o modelo e a normalização dos embeddings ---
model_name_embedding = "BAAI/bge-m3" # Mantendo BAAI/bge-m3
encode_kwargs_embedding = {'normalize_embeddings': True}

embeddings = HuggingFaceEmbeddings(
    model_name=model_name_embedding,
    encode_kwargs=encode_kwargs_embedding
)

# --- Criar ou carregar o Vectorstore com FAISS ---
faiss_index_path = './faiss_movie_index'

# Remover o índice FAISS existente para garantir um estado limpo se estiver recriando
if os.path.exists(faiss_index_path):
    print(f"Diretório '{faiss_index_path}' removido para recriação do índice FAISS.")
    shutil.rmtree(faiss_index_path)

vectorstore = None
if os.path.exists(faiss_index_path + "/index.faiss"):
    print(f"Carregando índice FAISS existente de '{faiss_index_path}'...")
    vectorstore = FAISS.load_local(faiss_index_path, embeddings, allow_dangerous_deserialization=True)
    print("Índice FAISS carregado com sucesso.")
else:
    print(f"Criando novo índice FAISS e persistindo documentos em '{faiss_index_path}'...")
    vectorstore = FAISS.from_documents(docs, embeddings)
    vectorstore.save_local(faiss_index_path)
    print("Índice FAISS criado e documentos persistidos com sucesso.")

/tmp/ipython-input-13-2296436168.py:9: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

Criando novo índice FAISS e persistindo documentos em './faiss_movie_index'...
Índice FAISS criado e documentos persistidos com sucesso.


In [ ]:
# --- Configuração da Cadeia de QA (RetrievalQA) ---
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(search_kwargs={"k": 5}),
    chain_type_kwargs={"prompt": prompt_template}, # Usando o prompt sem histórico
    return_source_documents=True
)

# --- Função de Resposta Principal (sem memória) ---
def responder(pergunta):
    # Procura trechos similares nos documentos de filmes
    similares = vectorstore.similarity_search(pergunta, k=5)

    # Critério de confiança: se não houver trechos próximos, nega a resposta
    if not similares or len(similares) == 0:
        return "Desculpe, não encontrei informações relevantes sobre isso nos filmes e séries que conheço. Tente perguntar sobre um título específico da minha base de dados."

    # Executa a cadeia de QA para obter a resposta
    resultado = qa_chain.invoke({"query": pergunta})
    return resultado["result"]

# A função de resposta para o Gradio.ChatInterface é simplificada
def chat_responder(message, history):
    # Apenas passa a mensagem para a função responder, sem se preocupar com o histórico
    return responder(message)

In [ ]:
# --- Interface Gradio (Formato de Chat) ---
with gr.Blocks() as demo:
    gr.Markdown("## 🎬 Main, seu Assistênte Virtual de Filmes e Série")

    gr.ChatInterface(
        fn=chat_responder,
        chatbot=gr.Chatbot(
            height=500,
            type="messages",
            value=[
                {"role": "assistant", "content": "Olá! Sou Main seu assistente de filmes e séries. Pergunte-me sobre sinopses, elenco, diretores ou qualquer outra coisa que queira saber sobre os filmes e séries que conheço!"}
            ]
        ),
        textbox=gr.Textbox(placeholder="Pergunte sobre um filme ou série...", container=False, scale=7),
        theme="soft",
        cache_examples=False,
    )

demo.launch(share=True)

/usr/local/lib/python3.11/dist-packages/gradio/chat_interface.py:322: UserWarning: The gr.ChatInterface was not provided with a type, so the type of the gr.Chatbot, 'messages', will be used.
  warnings.warn(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://c907067d3080739a13.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
